In [22]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

batch_size = 32
n_examples = 200
sequence_len = 30  # This is equivalent to time steps of the sequence in keras
sequence_size = 1
hidden_size = 50
nb_layers = 2
target_size = 1

# Generate noisy sine-wave
# When I want to complexify (many to one), just swap the x and y
NSAMPLE = 100000
f = 2 # the frequency of the signal
x_data = np.float32(np.arange(NSAMPLE))
r_data = np.float32(np.random.uniform(-0.2, 0.2, NSAMPLE))
y_data = np.float32(np.sin(2 * np.pi * f* (x_data/NSAMPLE)) + r_data)
# Build the training data
X = []
y = []
for i in range(0, y_data.shape[0], sequence_len):
    if i+sequence_len < y_data.shape[0]:
        X.append(x_data[i:i+sequence_len])
        y.append(y_data[i+sequence_len]) # next point
X = np.array(X)
y = np.array(y)

In [20]:
class my_rnn_1(nn.Module):
    def __init__(self, input_size=300, hidden_size=200, num_layers=3, output_size=1,
                 batch_size=10):
        super(my_rnn_1, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.batch_size = batch_size

        self.rnn = nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size,
                          num_layers=self.num_layers, batch_first=True)
        # The last layer is applied on the last output only of the RNN (not like
        # TimeDistributedDense in Keras)
        self.linear_layer = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input_sequence, hidden):
        out_rnn, hidden = self.rnn(input_sequence, hidden)
        in_linear = out_rnn[:, -1, :]
        final_output = self.linear_layer(in_linear)
        return final_output, hidden

    def init_hidden(self):
        return Variable(torch.zeros((self.num_layers, self.batch_size, self.hidden_size))).cuda()


def get_batch(X, Y, i, evaluation=False):
    global batch_size
    seq_len = min(batch_size, len(X) - 1 - i)
    data = X[i:i+seq_len, :]
    data = data.view(data.size(0), data.size(1), 1)
    target = Y[i+1:i+1+seq_len].view(-1, 1)
    return data, target

In [14]:
type(X)

numpy.ndarray

In [23]:
rnn = my_rnn_1(input_size=sequence_size, hidden_size=hidden_size, num_layers=3, batch_size=batch_size).cuda()
demo_target = Variable(torch.randn((batch_size, 1))).cuda()
X = Variable(torch.FloatTensor(X)).cuda()
y = Variable(torch.FloatTensor(y)).cuda()
loss_fn = nn.MSELoss()
# optimizer = optim.SGD(rnn.parameters(), lr=0.0001, momentum=0.9)
optimizer = optim.RMSprop(rnn.parameters())

for epoch in range(20):
    total_loss = 0
    for batch, i in enumerate(range(0, X.size(0) - 1, batch_size)):
        hidden = rnn.init_hidden()
        data, targets = get_batch(X, y, i)
        if data.size(0) < batch_size:
            break
        output, hidden = rnn(data, hidden)
        #print output
        optimizer.zero_grad()
        loss = loss_fn(output, targets)
        loss.backward()
        # loss.backward(retain_variables=True)
        optimizer.step()
        total_loss += loss.data[0]
        if i % 10 == 0:
            print("Loss = ", total_loss)
    print("Epoch " + str(epoch) + " -- loss = " + str(total_loss))
    print("-"*100)

Loss =  0.021808099001646042
Loss =  21.75482982955873
Loss =  21.83757818862796
Loss =  21.907587070018053
Loss =  21.99952644854784
Loss =  22.106887688860297
Loss =  22.217913476750255
Loss =  22.323810967616737
Loss =  22.396161892451346
Loss =  22.471900936216116
Loss =  22.567031182348728
Loss =  22.68315613642335
Loss =  22.771494332700968
Loss =  22.842347744852304
Loss =  22.946574336849153
Loss =  23.034023942425847
Loss =  23.142377232201397
Loss =  23.218812914565206
Loss =  23.3036791337654
Loss =  23.385761943645775
Loss =  23.482355510815978
Epoch 0 -- loss = 23.539296231232584
----------------------------------------------------------------------------------------------------
Loss =  0.026830799877643585
Loss =  0.11231172177940607
Loss =  0.1995922140777111
Loss =  0.2811197768896818
Loss =  0.43221230059862137
Loss =  0.6962181031703949
Loss =  1.246470533311367
Loss =  1.7153039649128914
Loss =  2.187428407371044
Loss =  2.8447896614670753
Loss =  3.215068232268095
L

Loss =  0.2171129733324051
Loss =  0.3266754485666752
Loss =  0.5639862567186356
Loss =  0.9728029780089855
Loss =  1.5526726208627224
Loss =  2.216251101344824
Loss =  2.50916600599885
Loss =  2.6993693597614765
Loss =  2.9997731745243073
Loss =  3.2339473105967045
Loss =  3.4740934148430824
Loss =  3.6572127044200897
Loss =  3.8333043325692415
Loss =  4.8143255058676
Loss =  5.996345380321145
Loss =  6.6784595269709826
Loss =  6.876622445881367
Loss =  7.107344675809145
Loss =  7.1962796887382865
Loss =  7.274148617871106
Epoch 12 -- loss = 7.328880170360208
----------------------------------------------------------------------------------------------------
Loss =  0.025061480700969696
Loss =  0.13070367462933064
Loss =  0.3511202968657017
Loss =  0.6086595095694065
Loss =  1.1399539969861507
Loss =  2.2180050425231457
Loss =  3.048817526549101
Loss =  3.691783893853426
Loss =  3.9601178243756294
Loss =  4.055501759052277
Loss =  4.201794940978289
Loss =  4.423586176708341
Loss =  4.